In [4]:
import numpy as np
import jax
import jax.numpy as jnp
import numpy as np
import openmm.app as app
import openmm.unit as unit
import openmm as mm
from dmff import Hamiltonian, NeighborListFreud
from tqdm import tqdm

In [ ]:
path = '/Users/arminsh/Documents/GADES/examples/Alanine Dipeptide/'
prmtop = app.AmberPrmtopFile(f"{path}/alanine-dipeptide.prmtop")
inpcrd = app.AmberInpcrdFile(f"{path}/alanine-dipeptide.inpcrd")

## Testing to see if DMFF is comparable to OpenMM

### DMFF

In [27]:
ff = Hamiltonian(f"{path}/protein.ff14SB.xml")

potentials = ff.createPotential(prmtop.topology, nonbondedMethod=app.NoCutoff, useDispersionCorrection=False)
params = ff.getParameters()
positions = jnp.array(inpcrd.getPositions(asNumpy=True).value_in_unit(unit.nanometer))

box = jnp.array([[20.0, 0.0, 0.0], [0.0, 20.0, 0.0], [0.0, 0.0, 20.0]])

nbList = NeighborListFreud(box, 3, potentials.meta["cov_map"]) # maybe remove the box?
nbList.allocate(positions)
pairs = nbList.pairs
efunc_all = potentials.getPotentialFunc()
dmff_e = efunc_all(positions, box, pairs, ff.paramset)

print("DMFF Pot E: ", dmff_e)

DMFF Pot E:  -55.794260834909345


### OpenMM

In [ ]:
forcefield = app.ForceField(f"{path}/protein.ff14SB.xml")
system = forcefield.createSystem(prmtop.topology, nonbondedMethod=app.NoCutoff)
integrator = mm.VerletIntegrator(0.001*unit.picoseconds)
platform = mm.Platform.getPlatformByName('Reference')
simulation = app.Simulation(prmtop.topology, system, integrator, platform)
simulation.context.setPositions(inpcrd.positions)

state = simulation.context.getState(getEnergy=True, getVelocities=True, getForces=True, getPositions=True)
energy = state.getPotentialEnergy()
print("OpenMM Pot E: ", energy)

OpenMM Pot E:  -55.794262524057814 kJ/mol


## Computing grad

```
# next two not needed if box, pairs, ff.paramset are constant
state_fnc = lambda pos: [pos, box, pairs, ff.paramset]
sample_state = state_fnc(positions)
```

In [37]:
params = ff.paramset # easy for implemenentation - must not change
efunc_pos = lambda pos: efunc_all(pos.reshape(-1, 3), box, pairs, ff.paramset) # for easy implementation 

grad_vec = jax.grad(efunc_pos)
force = grad_vec(positions.flatten())
print("Forces: \n", force)
force.shape

Forces: 
 [-1.71865366e+02 -3.18532912e+01  6.93394280e-01 -2.69338193e+02
 -3.51439108e+02 -2.29458474e+00 -5.28602801e+00  4.40961711e+01
  4.26381335e+01 -6.79627810e+00  4.33137449e+01 -4.09523736e+01
  4.52549770e+02  2.17764116e+02 -2.35854039e+02  6.63423153e+02
  4.01344132e+02 -3.88141697e+02  9.49967236e+01  7.69630908e+02
  4.01534867e+02 -4.90775901e+01 -4.57633388e+01  7.38115682e+00
 -4.28528076e+02 -3.13662037e+02 -1.89956133e+02  1.74440184e+01
  8.66267050e+00 -4.23307622e+01  1.04952670e+01  1.14678495e+02
  3.50646113e+02 -1.77497554e+01  2.76645948e+01  6.28447219e+01
 -8.25922834e+01  2.01080786e+02  1.30534383e+02 -4.08666513e+02
 -2.61975927e+02  3.33848622e+02  5.80858470e+01 -1.93137316e+01
 -2.68883324e+02  2.16593345e+02  4.39660829e+01  1.80234687e+01
  6.44641858e+01 -3.42606470e+02 -1.88895749e+02  1.40152566e+02
  1.03662316e+02  6.88386192e+00 -2.76119653e+02 -2.02536969e+02
  1.60757282e-01 -3.92006329e+01 -2.84702293e+02  9.29432738e-01
  1.83045552e+0

(66,)

## Computing hessians

In [53]:
def hessian_vec(positions):
  return jax.hessian(efunc_pos)(positions)

hess = hessian_vec(positions.flatten())
print(hess.shape)

# Compute the eigenvalues
w, v = jnp.linalg.eigh(hess)
w_idx = w.argsort()
w = w[w_idx]
v = v[:, w_idx]

(66, 66)


In [56]:
print("Eigevalues: \n", w)

Eigevalues: 
 [-1.20515267e+03 -9.26272741e+02 -4.90856230e+02 -2.64922500e+02
 -2.03726813e-10 -1.68236368e-11  7.27595761e-12  8.99995240e+01
  5.81406863e+02  1.33665166e+03  1.42352943e+03  2.31945079e+03
  3.55055531e+03  6.87779904e+03  1.29420881e+04  1.86465504e+04
  2.13280496e+04  2.61972148e+04  3.01487295e+04  3.29005454e+04
  3.99983779e+04  4.44728530e+04  4.72988656e+04  5.52189768e+04
  5.93176021e+04  6.03004173e+04  6.65167672e+04  7.09974092e+04
  7.32939158e+04  7.91970571e+04  8.20116199e+04  8.41032428e+04
  8.87786120e+04  9.57600111e+04  9.70146117e+04  1.00729146e+05
  1.07994144e+05  1.13327461e+05  1.30702156e+05  1.63781157e+05
  1.81842951e+05  1.93055695e+05  2.02625223e+05  2.21783957e+05
  2.37284932e+05  2.48731830e+05  2.62556336e+05  3.43251128e+05
  4.13542305e+05  5.07461558e+05  6.13608296e+05  6.75015846e+05
  7.02230832e+05  7.53955012e+05  7.58749080e+05  7.72377671e+05
  7.81461364e+05  7.92651267e+05  8.28555866e+05  8.46072878e+05
  9.4160005

In [8]:
def null_force(positions):
    return np.zeros_like(positions)

# BAOAB integrator
# https://en.wikipedia.org/wiki/Langevin_dynamics
# [p. 54] B. Leimkuhler and C. Matthews (2013) “Stochastic Numerical Methods for Molecular Sampling,” Applied Mathematics Research eXpress, Vol. 2013, No. 1, pp. 34–56 https://dx.doi.org/10.1093/amrx/abs010

def baoab_langevin_integrator(positions, velocities, forces_u, forces_b, mass, gamma, dt, kBT, force_function_u, force_function_b, n_steps=1):
    """
    BAOAB Langevin integrator based on Leimkuhler and Matthews (2013).

    Parameters:
        positions (np.ndarray): Initial positions (shape: [D, ], where D is dimensionality).
        velocities (np.ndarray): Initial velocities (shape: [D, ]).
        forces_u (np.ndarray): Initial unbiased forces (shape: [D, ]).
        forces_b (np.ndarray): Initial biased forces (shape: [D, ]).
        mass (float): Mass of the particles (scalar).
        gamma (float): Friction coefficient. (scalar).
        dt (float): Time step. (scalar).
        n_steps (int): Number of simulation steps. (scalar).
        kBT (float): Thermal energy (\(k_B T\)). (scalar).
        force_function_u (callable): Function to compute unbiased forces given positions (returns forces of shape [D, ]).
        force_function_b (callable): Function to compute biased forces given positions (returns forces of shape [D, ]).

    Returns:
        positions (np.ndarray): New positions (shape: [D, ]).
        velocities (np.ndarray): New velocities (shape: [D, ]).
        forces_u (np.ndarray): Unbiased forces at new position (shape: [D, ]).
        forces_b (np.ndarray): Biased forces at new position (shape: [D, ]).
    """
    dim = positions.shape

    # Precompute constants
    c1 = np.exp(-gamma * dt)
    c3 = np.sqrt(kBT * (1 - c1**2))
    inv_mass = np.reciprocal(mass)
    inv_mass_sqrt = np.reciprocal(np.sqrt(mass))

    for step in range(n_steps):

        # Step B (First half-step momentum update)
        forces = forces_u + forces_b
        velocities += 0.5 * dt * inv_mass * forces

        # Step A (Half-step position update)
        positions += 0.5 * dt * velocities

        # Step O (Thermostat and randomization)
        random_force = np.random.normal(size=(dim))
        velocities = c1 * velocities + c3 * inv_mass_sqrt * random_force

        # Step A (Second half-step position update)
        positions += 0.5 * dt * velocities

        # Step B (Second half-step momentum update)
        forces_u = force_function_u(positions)
        forces_b = force_function_b(positions)
        forces = forces_u + forces_b
        velocities += 0.5 * dt * inv_mass * forces

    return positions, velocities, forces_u, forces_b

def gad_force_vec(position, kappa=0.9):
    '''
    More efficient way to do this by passing forces_u in addition to position so don't have to double compute force_u,
    but annoying to use like this within BAOAB function
    
    Also annoying to hard-code kappa=XX into this function, but again annoying not to do so within BAOAB
    
    Should probably write custom BAOAB to accommodate
    '''
    
    # checking kappa is softening parameter
    assert 0. <= kappa <= 1.
        
    # unbiased forces
    forces_u = grad_vec(position)
    
    # biased forces (softened by kappa)
    hess = hessian_vec(position)
    # need to check if this is correct
    n_atoms = hess.shape[0] 
    n_dims = hess.shape[1] 
    hess_2d = hess.reshape(n_atoms * n_dims, n_atoms * n_dims)
    
    # Compute the eigenvalues
    w, v = jnp.linalg.eigh(hess_2d)
    n = v[:,0]
    # Normalize the eigenvector
    n = n / jnp.linalg.norm(n)
    
    # Reshape the forces_u to match the shape of n
    forces_u_flat = forces_u.reshape(-1)  # Flatten forces_u to (n_atoms * n_dims,)
    
    # Compute the biased forces
    forces_b_flat = -jnp.dot(n, forces_u_flat) * n  # Shape: (n_atoms * n_dims,)
    
    # Reshape the biased forces back to (n_atoms, n_dims)
    forces_b = forces_b_flat.reshape(n_atoms, n_dims)
    
    # Apply the softening parameter kappa
    forces_b *= kappa
    
    return forces_b

In [9]:
# Parameters
velocities = np.zeros_like(positions)
start_state = state_fnc(positions)
forces_u = grad_vec(positions)
forces_b = null_force(positions)
mass = mass_adp.reshape(-1,1) # needs to be mass of ADP - converting it into column vector
gamma = 1.0
dt = 0.001
n_steps = 10000
kBT = 2.494  # Thermal energy kJ/mol
# gad_force_vec(positions) # able to get gad forces

In [28]:
# forces_u + gad_force_vec(positions)
# prepare storage arrays
traj_p = np.full((n_steps+1, *positions.shape), np.nan)
traj_v = np.full((n_steps+1, *positions.shape), np.nan)
traj_fu = np.full((n_steps+1, *positions.shape), np.nan)
traj_fb = np.full((n_steps+1, *positions.shape), np.nan)

traj_p[0,:] = positions
traj_v[0,:] = velocities
traj_fu[0,:] = forces_u
traj_fb[0,:] = forces_b

In [29]:
# Run GAD
for i in tqdm(range(1,n_steps+1)):
    
    positions, velocities, forces_u, forces_b = baoab_langevin_integrator(
    positions, velocities, forces_u, forces_b, mass, gamma, dt, kBT, grad_vec, gad_force_vec
    )
    
    traj_p[i,:] = positions
    traj_v[i,:] = velocities
    traj_fu[i,:] = forces_u
    traj_fb[i,:] = forces_b

    # write a condition to save traj every 100 steps
    
    if i%100 == 0:
        traj_p_np = np.array(traj_p)
        np.save('traj_p.npy',traj_p_np)

# Convert trajectory to numpy array for analysis
traj_p = np.array(traj_p)
traj_v = np.array(traj_v)
traj_fu = np.array(traj_fu)
traj_fb = np.array(traj_fb)

  4%|████▉                                                                                                         | 448/10000 [01:40<35:42,  4.46it/s]


KeyboardInterrupt: 

In [ ]:
# def runMD(topfile, pdbfile, trajfile, length):
#     try:
#         os.remove("Lig_500.top")
#     except:
#         pass
#     top_prm = parmed.load_file(topfile)
#     top_500 = top_prm * 500
#     top_500.save("Lig_500.top")
#     pdb = app.PDBFile(pdbfile)
#     top = app.GromacsTopFile("Lig_500.top")
#     top.topology.setPeriodicBoxVectors(pdb.topology.getPeriodicBoxVectors())
#     system = top.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds, hydrogenMass=3*unit.dalton)
#     for force in system.getForces():
#         if isinstance(force, mm.NonbondedForce):
#             force.setUseDispersionCorrection(False)
#     system.addForce(mm.MonteCarloBarostat(1.0*unit.bar, 300*unit.kelvin, 25))
#     integ = mm.LangevinIntegrator(300*unit.kelvin, 1/unit.picosecond, 2.5*unit.femtosecond)
#     simulation = app.Simulation(top.topology, system, integ)
#     simulation.reporters.append(app.StateDataReporter(sys.stdout, 400, time=True, potentialEnergy=True, temperature=True, density=True, speed=True, remainingTime=True, totalSteps=int(length) * 400))
#     simulation.reporters.append(app.DCDReporter(trajfile, 400))
#     simulation.context.setPositions(pdb.getPositions())
#     simulation.minimizeEnergy(maxIterations=200)
#     simulation.step(int(length) * 400)
#     os.remove("Lig_500.top")

In [10]:
# forces_u + gad_force_vec(positions)
# prepare storage arrays
traj_p = np.full((n_steps+1, *positions.shape), np.nan)
traj_v = np.full((n_steps+1, *positions.shape), np.nan)
traj_fu = np.full((n_steps+1, *positions.shape), np.nan)
traj_fb = np.full((n_steps+1, *positions.shape), np.nan)

traj_p[0,:] = positions
traj_v[0,:] = velocities
traj_fu[0,:] = forces_u
traj_fb[0,:] = forces_b

# Run the integrator
for i in tqdm(range(1,n_steps+1)):
    
    positions, velocities, forces_u, forces_b = baoab_langevin_integrator(
    positions, velocities, forces_u, forces_b, mass, gamma, dt, kBT, grad_vec, null_force
    )
    
    traj_p[i,:] = positions
    traj_v[i,:] = velocities
    traj_fu[i,:] = forces_u
    traj_fb[i,:] = forces_b
    if i%100 == 0:
        traj_p_np = np.array(traj_p)
        np.save('traj_p_unbiased.npy',traj_p_np)
    

# Convert trajectory to numpy array for analysis
traj_p = np.array(traj_p)
traj_v = np.array(traj_v)
traj_fu = np.array(traj_fu)
traj_fb = np.array(traj_fb)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [09:47<00:00, 17.02it/s]


array([[[ 2.00000107e-01,  1.00000001e-01, -1.30000004e-07],
        [ 2.00000107e-01,  2.09000006e-01,  9.99999994e-09],
        [ 1.48626402e-01,  2.45384902e-01,  8.89824033e-02],
        ...,
        [ 4.81857598e-01,  8.64773512e-01,  1.04000003e-06],
        [ 6.35979831e-01,  8.64773154e-01,  8.89828280e-02],
        [ 6.35978997e-01,  8.64773512e-01, -8.89818668e-02]],

       [[ 1.99903399e-01,  1.00017600e-01,  3.02426361e-05],
        [ 1.99974716e-01,  2.08990544e-01, -6.25230177e-06],
        [ 1.48607701e-01,  2.45344073e-01,  8.90382677e-02],
        ...,
        [ 4.81822222e-01,  8.64603996e-01, -5.40142173e-06],
        [ 6.36004746e-01,  8.64748836e-01,  8.90221223e-02],
        [ 6.35976434e-01,  8.64675999e-01, -8.89606252e-02]],

       [[ 1.99667603e-01,  1.00008927e-01,  6.04168417e-05],
        [ 1.99908674e-01,  2.08967775e-01, -1.94850072e-05],
        [ 1.48589358e-01,  2.45280415e-01,  8.91810954e-02],
        ...,
        [ 4.81780648e-01,  8.64108980e-01,